# Renewable Energy Heat

- Juan Manuel Boullosa Novo
- Alberto Jimenez Haro
- Emre Atalan

In [ ]:
from CoolProp.CoolProp import PropsSI
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import cm
import _roomClass as rc # Custom class required ./_roomClass.py

Function definition for cooling demand

In [ ]:
def calculate_cooling_demand(location, coldRoom):
    cooling_demand = location.calculate_yearly_cooling_demand(coldRoom)
    rc.Plotter.plot_cooling_demand(location.months, cooling_demand, location.average_temperatures)
    print("Heating demand for each month:")
    for month, demand in zip(location.months, cooling_demand):
        print(f"{month}: {demand:.2f} kW")
    print(f"Average heating demand: {np.mean(cooling_demand):.2f} kW")
    return cooling_demand

We set the parameters for the location and the room and create the instances.

In [ ]:
average_temperature_hamburg = [1.7, 2, 4.5, 9.1, 13.3, 16.3, 18.5, 18.1, 14.9, 10.5, 6, 3] # in °C [Source: https://en.climate-data.org/europe/germany/hamburg/hamburg-69/]
desired_freezer_temperature = -18 # in °C
coldroom_dimensions = [4, 3, 2] # in meters
floor_material = "Concrete"
wall_material = "Polyurethane foam"
ceiling_material = "Polyurethane foam"

coldRoom = rc.Room(coldroom_dimensions, floor_material, wall_material, ceiling_material, 50, 15, 20, desired_freezer_temperature)    
hamburg = rc.ClimateData(average_temperature_hamburg)

We calculate the yearly cooling demand

In [ ]:
yearly_heating_demand = calculate_cooling_demand(hamburg, coldRoom)

Let's create the functions that calculates the evaporators heat, the power and the cop:

In [ ]:
def cop_and_heat_calculations(fluid, temp_2, temp_4, efficiency_compressor, heat_evap):
    s_2 = PropsSI("S", "T", temp_2, "Q", 1, fluid)
    h_2 = PropsSI("H", "T", temp_2, "Q", 1, fluid)
    press_2 = PropsSI("P", "T", temp_2, "Q", 1, fluid)
    press_4 = PropsSI("P", "T", temp_4, "Q", 0, fluid)
    press_3 = press_4 # pressure remains same after compressor / before expansion

    h_3s = PropsSI("H", "S", s_2, "P", press_3, fluid)
    h_3 = h_2 + (h_3s - h_2) / efficiency_compressor # enthalpy considering compressor efficiency

    h_4 = PropsSI("H", "T", temp_4, "Q", 0, fluid)
    h_1 = h_4 # enthalpy remains constant in expansion. refer to ph graph
    press_1 = press_2 # pressure remains constant in evaporation.

    # no calculamos s_1, s_3, s_4, t_1, t_3

    m = heat_evap / abs(h_2 - h_1) # mass flow rate kg/s. Are these enthalpies correct? or should we get h_4, h_3
    power = m * abs(h_3 - h_2) # compressor power
    cop = abs(heat_evap) / power
    heat_cond = abs(heat_evap) - power

    cop_carnot = abs(temp_2 / (temp_2 - temp_4))
    efficiency_hp = cop / cop_carnot
    
    return m, power, cop, heat_cond, cop_carnot, efficiency_hp

def steady_state_cop_analysis(fluid_list, temp_list, fixed_temp, efficiency_compressor, heat_evap, mode='evap'):
    cop_list = []
    carnot_cop_list = []
    for fluid in fluid_list:
        for temp in temp_list:
            if mode == 'evap':
                _, _, cop, _, cop_carnot, efficiency_hp = cop_and_heat_calculations(fluid, temp, fixed_temp, efficiency_compressor, heat_evap)
            else:
                _, _, cop, _, cop_carnot, efficiency_hp = cop_and_heat_calculations(fluid, fixed_temp, temp, efficiency_compressor, heat_evap)
            cop_list.append(cop)
            carnot_cop_list.append(cop_carnot)
    return cop_list, carnot_cop_list

def seasonal_cop_and_heat_calculations(fluid, evap_temp, cond_temp, efficiency_compressor, months, heat_list):
    cop_list = []
    for month, heat_evap in zip(months, heat_list):
        m_month, power_month, cop_month, heat_cond_month, cop_carnot_month, efficiency_hp_month = cop_and_heat_calculations(fluid, evap_temp, cond_temp, efficiency_compressor, heat_evap)
        cop_list.append(cop_month)
        heat_list.append(heat_cond_month)
    return cop_list, heat_list


Let's select some of the necessary parameters for our study:

In [ ]:
outside_temp = hamburg.average_yearly_temp + 273.15 # Average temperature in Hamburg in Kelvin
desired_temp = -18 + 273.15 # C Desired temperatures

evap_temp = desired_temp - 10
cond_temp = outside_temp + 20

evap_temp_list = np.arange(evap_temp - 10 , evap_temp + 10 , 2) # Range of evaporating temperatures
cond_temp_list = np.arange(cond_temp - 10 , cond_temp + 10 , 2) # Range of condensing temperatures

fluid_list = ["R717", "R134a", "R290", "R600a", "R404a", "R407c", "R410a"]
efficiency_compressor = 0.8 # Efficiency assumed to be 80%
heat_evap = abs(coldRoom.total_power_demand(outside_temp)) # Nominal yearly heat power demand (W)

Steady State COP Analysis

In [ ]:
steady_state_cop_list_evap, steady_state_carnot_cop_list_evap = steady_state_cop_analysis(fluid_list, evap_temp_list, cond_temp, efficiency_compressor, heat_evap, 'evap')
steady_state_cop_list_cond, steady_state_carnot_cop_list_cond = steady_state_cop_analysis(fluid_list, cond_temp_list, evap_temp, efficiency_compressor, heat_evap, 'cond')

In [ ]:
refrigerants = ["R717 (Ammonia)", "R134a", "R290 (Propane)", "R600a (Isobutane)", "R404a", "R407c", "R410a"]

# Plotting
fig, ax = plt.subplots(1, 2, figsize=(15, 5))

# Plot steady state COP
for i, refrigerant in enumerate(refrigerants):
    ax[0].plot(evap_temp_list - 273.15, steady_state_cop_list_evap[i*len(evap_temp_list):(i+1)*len(evap_temp_list)], label=refrigerant)
    ax[1].plot(cond_temp_list - 273.15, steady_state_cop_list_cond[i*len(cond_temp_list):(i+1)*len(cond_temp_list)], label=refrigerant)

# Plot Carnot COP
ax[0].plot(evap_temp_list - 273.15, steady_state_carnot_cop_list_evap[:len(evap_temp_list)], label="Carnot COP", linestyle="--")
ax[1].plot(cond_temp_list - 273.15, steady_state_carnot_cop_list_cond[:len(cond_temp_list)], label="Carnot COP", linestyle="--")

# Plot chosen temperatures
ax[0].axvline(x=evap_temp - 273.15, color='k', linestyle='dotted', label="Chosen evap. temp.")
ax[1].axvline(x=cond_temp - 273.15, color='k', linestyle='dotted', label="Chosen cond. temp.")

ax[0].set_xlabel("Evaporating temperature (°C)")
ax[0].set_ylabel("COP")
ax[0].set_title("COP vs Evaporating temperature")
ax[0].legend()
ax[0].grid()

ax[1].set_xlabel("Condensing temperature (°C)")
ax[1].set_ylabel("COP")
ax[1].set_title("COP vs Condensing temperature")
ax[1].legend()
ax[1].grid()

Now, let's plot both the condensated T° and the evaporated T° in the same 3D graph to compare results:

In [ ]:
param_cop = pd.DataFrame(
    index=evap_temp_list, 
    columns=cond_temp_list
)

for ev_temp in evap_temp_list:
    _, _, cop, _, _, _ = cop_and_heat_calculations(
        fluid_list, 
        ev_temp, 
        cond_temp_list, 
        efficiency_compressor, 
        heat_evap
    )
    param_cop.loc[ev_temp] = cop

param_cop

# Plot 3D Figure:
fig, ax = plt.subplots(1, figsize=(12, 7.5), subplot_kw={"projection": "3d"})

X, Y = np.meshgrid(cond_temp_list - 273.15, evap_temp_list - 273.15)

surf = ax.plot_surface(X, Y, param_cop.values, cmap=cm.coolwarm, linewidth=0, antialiased=False)

ax.set_xlabel("Condensator temperature [°C]")
ax.set_ylabel("Evaporator temperature [°C]")
ax.set_zlabel("COP")
ax.view_init(elev=30, azim=225)
ax.set_box_aspect(aspect=None, zoom=0.9)
fig.colorbar(surf, shrink=0.5, aspect=15)
plt.tight_layout()

Let's try plot the P-H Diagram:

In [ ]:

def press_and_enthalpy_calc(fluid, temp_2, temp_4, efficiency_compressor, heat_evap):
    s_2 = PropsSI("S", "T", temp_2, "Q", 1, fluid)
    h_2 = PropsSI("H", "T", temp_2, "Q", 1, fluid)
    press_2 = PropsSI("P", "T", temp_2, "Q", 1, fluid)
    press_4 = PropsSI("P", "T", temp_4, "Q", 0, fluid)
    press_3 = press_4 # pressure remains same after compressor / before expansion

    h_3s = PropsSI("H", "S", s_2, "P", press_3, fluid)
    h_3 = h_2 + (h_3s - h_2) / efficiency_compressor # enthalpy considering compressor efficiency

    h_4 = PropsSI("H", "T", temp_4, "Q", 0, fluid)
    h_1 = h_4 # enthalpy remains constant in expansion. refer to ph graph
    press_1 = press_2 # pressure remains constant in evaporation.

    # no calculamos s_1, s_3, s_4, t_1, t_3

    m = heat_evap / abs(h_2 - h_1) # mass flow rate kg/s. Are these enthalpies correct? or should we get h_4, h_3
    power = m * abs(h_3 - h_2) # compressor power
    cop = abs(heat_evap) / power
    heat_cond = abs(heat_evap) - power
    
    return h_1, h_2, h_3, h_4, press_1, press_2, press_3, press_4

h_1, h_2, h_3, h_4, press_1, press_2, press_3, press_4 = press_and_enthalpy_calc(
    fluid_list, evap_temp, cond_temp, efficiency_compressor, heat_evap
)

p_range = np.geomspace(PropsSI("P", "T", 273.15 -30, "Q", 0, fluid_list), PropsSI("PCRIT", fluid_list))
boiling_line = PropsSI("H", "P", p_range, "Q", 0, fluid_list) / 1e3
dew_line = PropsSI("H", "P", p_range, "Q", 1, fluid_list) / 1e3

fig, ax = plt.subplots(1, figsize=(12, 6))

ax.set_yscale("log")
ax.plot(boiling_line, p_range, color="#000")
ax.plot(dew_line, p_range, color="#000")

ax.scatter([h / 1e3 for h in [h_1, h_2, h_3, h_4]], [press_1, press_2, press_3, press_4])
ax.plot([h / 1e3 for h in [h_1, h_2, h_3, h_4, h_1]], [press_1, press_2, press_3, press_4, press_1])

ax.set_ylabel("pressure in Pa")
ax.set_xlabel("enthalpy in kJ/kg")

_x_lims = ax.get_xlim()
_y_lims = ax.get_ylim()


plt.tight_layout()